In [ ]:
# Importing the Packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


from pathlib import Path

In [ ]:
# Installing Pytorch 1.6 for FastAI v2

!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# Installing FastAI v2

!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *

In [ ]:
# Checking if cuda is available to make use of GPU

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

In [ ]:
# Seed everything

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(42)

In [ ]:
path = Path("../input/aptos2019-blindness-detection")
path.ls()

In [ ]:
# Let's have a look at the training data

train_df = pd.read_csv(path/"train.csv")
print(train_df.shape)
train_df.head()

In [ ]:
# The test data does not have any labels that we are going to predict

test_df = pd.read_csv(path/"test.csv")
print(test_df.shape)
test_df.head()

In [ ]:
train_path = path/"train_images"
train_path.ls()

Number of images in train folder is same as number of records in the csv file. We can go ahead with creating DataBunch.

In [ ]:
# Let's have a look at some of the images in the folder

fig = plt.figure(figsize = (10,10))
rows = 3
columns = 3
for i in range(1, rows*columns+1):
    img = Image.open(train_path.ls()[i])
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    
plt.show()

In [ ]:
train_df['diagnosis'].value_counts()

There is some imbalance in the classes. For now, we will proceed normally.
<br> We may try some balancing in the future experiments.

In [ ]:
LABEL_COLS = [0, 1, 2, 3, 4]

In [ ]:
def get_dls(size, bs):
    datablock = DataBlock(
                blocks = (ImageBlock, CategoryBlock(vocab = LABEL_COLS)),
                get_x = ColReader('id_code', pref = train_path, suff = '.png'),
                get_y = ColReader('diagnosis'),
                splitter = RandomSplitter(valid_pct = 0.2, seed= 42),
                item_tfms = Resize(size)
    )
    
    return datablock.dataloaders(train_df, bs= bs)


In [ ]:
dls = get_dls(224, 64)

In [ ]:
dls.show_batch()

In [ ]:
dls.valid.show_batch()

### Modelling

A simple way to think this is that Cohen’s Kappa is a quantitative measure of reliability for two raters that are rating the same thing, corrected for how often that the raters may agree by chance.<br>
<br>
Cohen suggested the Kappa result be interpreted as follows: values ≤ 0 as indicating no agreement and 0.01–0.20 as none to slight, 0.21–0.40 as fair, 0.41– 0.60 as moderate, 0.61–0.80 as substantial, and 0.81–1.00 as almost perfect agreement.


In [ ]:
ARCH = densenet121
quadratic_kappa = CohenKappa(weights = 'quadratic')

In [ ]:
learn = cnn_learner(dls, ARCH, metrics = [accuracy, quadratic_kappa])

In [ ]:
learn.loss_func

In [ ]:
learn.lr_find()

In [ ]:
lr = 3e-3
learn.fit_one_cycle(10, lr)
learn.recorder.plot_loss()

In [ ]:
learn.show_results(nrows=3, figsize=(6,8))

In [ ]:
# Plot Confusion Matrix

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(normalize = True)

In [ ]:
# Save the model

learn.save("densenet121-stage-1")
learn = learn.load("densenet121-stage-1")

**If you like it , please upvote :)**